In [4]:
from mltranslator.modules.inpainting.inpaintor import * 
from mltranslator import PROJECT_DIR
from PIL import Image
from matplotlib import pyplot as plt

In [5]:
my_inpaintor = Inpaintor()

The `test2_mask_gt.jpg` is manually drawn for testing purpose.

In [6]:
image = Image.open(f'{PROJECT_DIR}/dataset/test/test2.jpg')
mask = Image.open(f'{PROJECT_DIR}/dataset/test/test2_mask_gt.jpg')

# plot image

fig, _ = plt.subplots(1,2, figsize=(8,8))
fig.axes[0].imshow(image)
fig.axes[0].axis("off")
fig.axes[0].set_title("original image")
fig.axes[1].imshow(mask)
fig.axes[1].axis("off")
fig.axes[1].set_title("mask")
plt.show()

In [7]:
inpaint_mask, inpaint_result = my_inpaintor.inpaint_custom(image, np.array(mask))

In [8]:
inpaint_result_pil = Image.fromarray(inpaint_result)
fig, _ = plt.subplots(1, 2, figsize=(8, 8))
fig.axes[0].imshow(image)
fig.axes[0].axis("off")
fig.axes[0].set_title("original image")
fig.axes[1].imshow(inpaint_result_pil)
fig.axes[1].axis("off")
fig.axes[1].set_title("Inpaint image")
plt.show()

In [9]:
mask_v2, inpaint_result_v2 = my_inpaintor.inpaint_v2(image)

mask_v2_pil = Image.fromarray(mask_v2)
inpaint_result_v2_pil = Image.fromarray(inpaint_result_v2)

fig, _ = plt.subplots(1,3, figsize=(12,8))
fig.axes[0].imshow(image)
fig.axes[0].axis("off")
fig.axes[0].set_title("original image")
fig.axes[1].imshow(inpaint_result_v2_pil)
fig.axes[1].axis("off")
fig.axes[1].set_title("Inpaint image v2")
fig.axes[2].imshow(mask_v2_pil)
fig.axes[2].axis("off")
fig.axes[2].set_title("mask v2")
plt.show()

In [10]:
inpaint_result_v2_pil.size, mask_v2_pil.size

In [11]:
merged_image = Image.blend(image, mask_v2_pil.convert("RGB"), alpha=0.50)

fig, _ = plt.subplots(1,2, figsize=(12,8))
fig.axes[0].imshow(merged_image)
fig.axes[0].axis("off")
fig.axes[0].set_title("original image")
fig.axes[1].imshow(inpaint_result_v2_pil)
fig.axes[1].axis("off")
fig.axes[1].set_title("Inpaint image v2")
# fig.axes[2].imshow(Image.fromarray(mask_v2))
# fig.axes[2].axis("off")
# fig.axes[2].set_title("mask v2")
plt.show()
